In [1]:
import cv2

x_start, y_start, width, height = 100, 100, 200, 200  

def main():
    cap = cv2.VideoCapture(0) 
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return
    
    ret, frame1 = cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        return

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (21, 21), 0)

    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.GaussianBlur(gray2, (21, 21), 0)


        diff = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)

        roi = thresh[y_start:y_start+height, x_start:x_start+width]

        contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cv2.rectangle(frame2, (x_start, y_start), (x_start + width, y_start + height), (0, 255, 0), 2)

        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue  

            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame2, (x_start + x, y_start + y), (x_start + x + w, y_start + y + h), (0, 0, 255), 2)

        cv2.imshow("Frame", frame2)
        cv2.imshow("Thresh", thresh)
        cv2.imshow("ROI", roi)

        gray1 = gray2

        if cv2.waitKey(30) & 0xFF == 27:  #press "esc" to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
